# Få tak i værdata

In [9]:
from openai import OpenAI
import json

client = OpenAI()

messages = [
  {
    "role": "system",
    "content": "You are a weather reporter.",
  },
  {
    "role": "user",
    #"content": "Hvordan er været i Oslo?",
    "content": "Hvordan er været i Oslo? Og hva med Trondheim? Og hva er temperaturforskjellen mellom de to byene? Hva med Bergen?",
  }
]

tools = [
  {
    "type": "function",
    "function": {
      "name": "get_weather",
      "description": "Get the weather for a given location.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The location to get the weather for.",
          }
        },
        "required": ["location"],
      }
    }
  }
]

def get_weather(location):
  #import requests
  #response = requests.get(f"https://wttr.in/{location}?format=j1").json()
  #return json.dumps(response["current_condition"][0])
  return ["Sol og snø, 10 grader", "Lys, lyn og varme, 5 grader", "Regn og trist, -1 grad"][len(location) % 3]

tools_functions = {
  "get_weather": lambda args: get_weather(args["location"])
}

completion = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=messages,
    tools=tools,
)

i = 0
while True:
  if i > 10:
    print("Too many tool calls.")
    break
  i += 1

  if completion.choices[0].finish_reason == "stop":
    print()
    print("Response:")
    print(completion.choices[0].message.content)
    break

  if completion.choices[0].finish_reason == "tool_calls":
    tool_calls = []
    tool_results = []
    tool_calls_count = len(completion.choices[0].message.tool_calls)
    if tool_calls_count > 1:
      print(f"Going to execute {tool_calls_count} tool calls in parallel:")
    for call in completion.choices[0].message.tool_calls:
      tool_id = call.id
      tool_name = call.function.name
      tool_args = call.function.arguments
      print(f"Calling tool: \"{call.function.name}\" with parameters: " + str(tool_args).replace("\n", " ").replace("   ", " "))
      value = tools_functions[tool_name](json.loads(tool_args))
      tool_calls.append({
        "id": tool_id,
        "type": "function",
        "function": {
          "name": tool_name,
          "arguments": tool_args,
        }
      })
      tool_results.append({
          "role": "tool",
          "content": str(value),
          "tool_call_id": tool_id,
      })
      print("Sending tool return value back to GPT")
      print()

    messages.append({
        "role": "assistant",
        "tool_calls": tool_calls,
    })

    messages += tool_results

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        tools=tools,
        temperature=0,
    )


Going to execute 3 tool calls in parallel:
Calling tool: "get_weather" with parameters: {"location": "Oslo"}
Sending tool return value back to GPT

Calling tool: "get_weather" with parameters: {"location": "Trondheim"}
Sending tool return value back to GPT

Calling tool: "get_weather" with parameters: {"location": "Bergen"}
Sending tool return value back to GPT


Response:
Været i Oslo er lys, lyn og varme med en temperatur på 5 grader. I Trondheim er det sol og snø med en temperatur på 10 grader. Bergen har også sol og snø med en temperatur på 10 grader. 

Temperaturforskjellen mellom Oslo og Trondheim er 5 grader.
